In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# function to compute trajectory from dFdt
from ipynb.fs.defs.integration_dFdt import integration_dFdt

In [3]:
def gradient_flowmap(time, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid):
    
    dx = delta[0]
    dy = delta[1]
    
    xL = x-np.array([1, 0])*aux_grid*dx
    xLend = integration_dFdt(time, xL, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
        
    xR = x+np.array([1, 0])*aux_grid*dx
    xRend = integration_dFdt(time, xR, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
        
    xD = x-np.array([0, 1])*aux_grid*dy
    xDend = integration_dFdt(time, xD, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
        
    xU = x+np.array([0, 1])*aux_grid*dy
    xUend = integration_dFdt(time, xU, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
    
    gradFmap = np.zeros((2, 2, len(time)))
    
    for i in range(len(time)):
        
        gradFmap[0, 0, i] = (xRend[0, i]-xLend[0, i])/(2*aux_grid*dx)
        gradFmap[1, 0, i] = (xRend[1, i]-xLend[1, i])/(2*aux_grid*dx)
        
        gradFmap[0, 1, i] = (xUend[0, i]-xDend[0, i])/(2*aux_grid*dy)
        gradFmap[1, 1, i] = (xUend[1, i]-xDend[1, i])/(2*aux_grid*dy)
        
    return gradFmap